# Websearch 優缺點與應用策略

## 優點
- **多元化來源**：涵蓋範圍廣，能提供多角度資訊。
- **即時性**：能快速取得公開網頁上的最新內容。
- **靈活性**：適合需要「多來源比對」的問題。

## 缺點
- **碎片化**：資訊分散、格式不一，難以直接系統化使用。
- **品質參差不齊**：來源可靠度不同，可能存在錯誤或過時資訊。
- **限制與風險**：部分 API 或搜尋過程可能因政策、安全或授權而阻擋特定內容。

---

## 何時適合使用 Websearch
- 需要 **多角度觀點**（如新聞、論壇、社群資訊）。
- **開放探索**，對來源精確度要求不高。
- 無法透過單一可靠資料庫滿足需求時。

---

## 何時更適合使用特定來源
- **專門領域**：如 Wikipedia、Fandom Wiki（例如遊戲、小說、Warhammer 40k）。
- **結構化資料需求**：來源有規則的網址與內容組織，便於程式化檢索。
- **高可信度需求**：減少處理過多雜訊。

---

## API 使用注意事項
- **安全審查阻擋**：可能因涉及「不允許或敏感內容」而無法獲取公開資料。
- **授權與限制**：包含付費牆、Rate Limit、隱私規範等。
- **備援角色**：Websearch 可作為補充方案，但不一定是萬能解決方式。

---

## 總結
Websearch 提供了 **廣泛而多元的資訊**，但也帶來 **碎片化與品質問題**。  
若需求明確、可依靠結構化且可信的來源（如 Wikipedia、Fandom），應優先選用。  
若需要多角度、開放探索或無特定資料庫可依賴時，Websearch 才能發揮最大價值。


## 返無 歸一

- 假設你確定在某個來源肯定有資訊的時候，取得該來源的網址
- 使用第一周和第三周的技巧爬取網址的內容
- 透過LLM提取你要的訊息

In [ ]:
import os

os.chdir("../../../")

In [ ]:
from openai import OpenAI
from textwrap import dedent
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

from src.initialization import credential_init


def build_standard_chat_prompt_template(kwargs):

    messages = []
 
    if 'system' in kwargs:
        content = kwargs.get('system')
        prompt = PromptTemplate(**content)
        message = SystemMessagePromptTemplate(prompt=prompt)
        messages.append(message)  

    if 'human' in kwargs:
        content = kwargs.get('human')
        prompt = PromptTemplate(**content)
        message = HumanMessagePromptTemplate(prompt=prompt)
        messages.append(message)
        
    chat_prompt_template = ChatPromptTemplate.from_messages(messages)
    
    return chat_prompt_template

credential_init()

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

### 抽取Wikipedia的內容

In [ ]:
import requests

S = requests.Session()

query = "獵人中的念能力系統"

# Wikipedia語言選擇
URL = "https://ja.wikipedia.org/w/api.php"


PARAMS = {
    "action": "parse",
    # Wikipedia 頁面的 關鍵字
    "page": "HUNTER×HUNTER",
    "prop": "text",
    "format": "json"
}

HEADERS = {
    "User-Agent": "AI Tutorial Bot/1.0 (mengchiehling@gmail.com)"
}

R = S.get(url=URL, params=PARAMS, headers=HEADERS)

data = R.json()

使用 bs4 處理數據 

In [ ]:
from bs4 import BeautifulSoup

html_content = data['parse']['text']["*"]

soup = BeautifulSoup(html_content, "html.parser")

# 移除 style 和 script
for tag in soup(["style", "script"]):
    tag.decompose()

# 提取文字
text_content = soup.get_text(separator="\n")

# 清理空白與空行
cleaned_text = "\n".join(
    line.strip() for line in text_content.splitlines() if line.strip()
)

In [ ]:
# cleaned_text

慢慢寫Pydantic物件是可以實現的目標的，但是在應用中我們希望有更好的自動化: 根據使用者的需求自動生成物件

我們嘗試結合代碼生成來輔助完成目標

In [ ]:
code_example = dedent("""
    Name extraction: 

    from pydantic import BaseModel, Field
    from langchain_core.output_parsers import PydanticOutputParser

    
    class NameExtractor(BaseModel):
        name: str = Field(description="The returned name")

    output_parser = PydanticOutputParser(pydantic_object=NameExtractor)
    format_instructions = output_parser.get_format_instructions()
""")


system_template = dedent(f"""
    You are a highly skilled Python developer with expertise in large language model with Langchain framework. 
    Your task is to generate Python code to solve the user's problem.
    Leverage the Pydantic model. 
    Your response must contain only the Python code — no explanations, comments, or additional text.

    Here is an example of expected structure:

    {code_example}
""")

human_template = dedent("""{query}
                            Code:
                        """)


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"]}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

model = ChatOpenAI(model="gpt-4o-mini")

code_generation = chat_prompt_template|model|StrOutputParser()

In [ ]:
generated_code = code_generation.invoke({"query": query})

In [ ]:
print(generated_code)

### 代碼執行工具

In [ ]:
import re

from langchain_core.runnables import chain

@chain
def code_execution(code):
    
    match = re.findall(r"python\n(.*?)\n```", code, re.DOTALL)
    python_code = match[0]
    
    lines = python_code.strip()#.split('\n')
    # *stmts, last_line = lines

    local_vars = {}
    exec(lines, local_vars)

    return local_vars

In [ ]:
local_vars = code_execution.invoke(generated_code)

In [ ]:
human_template = dedent("""{query}
                            context:
                            {context}
                           output format instruction = {format_instruction} 
                        """)


input_ = {"system": {"template": "Return the result in traditional Chinese"},
          "human": {"template": human_template,
                    "input_variable": ["query"],
                    "partial_variables": {'format_instruction': local_vars["format_instructions"]}
                    }}

chat_prompt_template = build_standard_chat_prompt_template(input_)

model = ChatOpenAI(model="gpt-4o-mini")

pipeline = chat_prompt_template|model|local_vars['output_parser']

output = pipeline.invoke({"query": query,
                 "context": cleaned_text})

In [ ]:
query

In [ ]:
output